In [ ]:
find . -type f -name '#*#' -delete

In [ ]:
module purge
module load GCC/10.3.0
module load OpenMPI/4.1.1
module load GROMACS/2021.3-CUDA-11.3.1

# 23C3-v1

In [ ]:
sample=23C3_v1
WORKDIR=${ANALYSIS_DIR}/in/${sample}
cd ${WORKDIR}

## 1.1. Minimization

In [ ]:
init=step3_input
mini_prefix=step4.0_minimization
RUN_ID="EM"
./../../src/simulation_sbatch_script.sh --init_gro ${WORKDIR}/${init}.gro \
                                    --topol ${WORKDIR}/topol.top \
                                    --run_prefix ${mini_prefix} \
                                    --mdp ${WORKDIR}/step4.0_minimization.mdp \
                                    --output_dir ${WORKDIR}/../../out/${sample}/${mini_prefix} \
                                    --sample ${sample} \
                                    --workdir ${WORKDIR} \
                                    --lab "gdwanglab" \
                                    --job_name "em" \
                                    --run ${RUN_ID} \
                                    --index ${WORKDIR}/index.ndx
                                    
sbatch ${WORKDIR}/${sample}_${mini_prefix}_Run_${RUN_ID}.sbatch

## 1.2. Equilibration

In [ ]:
RUN_ID="NVT"
prod_prefix="md"
equi_prefix=step4.1_equilibration

./../../src/simulation_sbatch_script.sh   --init_gro ${WORKDIR}/../../out/${sample}/${mini_prefix}/Run_EM/${mini_prefix}_EM.gro \
                                    --topol ${WORKDIR}/topol.top \
                                    --run_prefix ${equi_prefix} \
                                    --mdp ${WORKDIR}/step4.1_equilibration.mdp \
                                    --output_dir ${WORKDIR}/../../out/${sample}/${equi_prefix} \
                                    --sample ${sample} \
                                    --workdir ${WORKDIR} \
                                    --lab "gdwanglab" \
                                    --job_name "equil" \
                                    --run_duration "80:00:00" \
                                    --run ${RUN_ID} \
                                    --index ${WORKDIR}/index.ndx

sbatch ${WORKDIR}/${sample}_${equi_prefix}_Run_${RUN_ID}.sbatch

## 1.3. MD

In [ ]:
md_prefix="md"
equi_prefix="step4.1_equilibration"
for chain in chain1 chain2 chain3
do
    RUN_ID="MD_${chain}"

    ./../../src/simulation_sbatch_script.sh \
        --init_gro ${WORKDIR}/../../out/${sample}/${equi_prefix}/Run_NVT/${equi_prefix}_NVT.gro \
        --topol ${WORKDIR}/topol.top \
        --run_prefix ${md_prefix} \
        --mdp ${WORKDIR}/step5_production.mdp \
        --output_dir ${WORKDIR}/../../out/${sample}/${md_prefix} \
        --sample ${sample} \
        --workdir ${WORKDIR} \
        --lab "gdwanglab" \
        --job_name "md" \
        --run_duration "250:00:00" \
        --run ${RUN_ID} \
        --index ${WORKDIR}/index.ndx

    sbatch ${WORKDIR}/${sample}_${md_prefix}_Run_${RUN_ID}.sbatch
done


# 2. Hu23C3-v1

In [ ]:
sample=Hu23C3_v1
WORKDIR=${ANALYSIS_DIR}/in/${sample}
cd ${WORKDIR}

## 2.1. Minimization

In [ ]:
init=step3_input
mini_prefix=step4.0_minimization
RUN_ID="EM"
./../../src/simulation_sbatch_script.sh --init_gro ${WORKDIR}/${init}.gro \
                                    --topol ${WORKDIR}/topol.top \
                                    --run_prefix ${mini_prefix} \
                                    --mdp ${WORKDIR}/step4.0_minimization.mdp \
                                    --output_dir ${WORKDIR}/../../out/${sample}/${mini_prefix} \
                                    --sample ${sample} \
                                    --workdir ${WORKDIR} \
                                    --lab "gdwanglab" \
                                    --job_name "em" \
                                    --run ${RUN_ID} \
                                    --index ${WORKDIR}/index.ndx

sbatch ${WORKDIR}/${sample}_${mini_prefix}_Run_${RUN_ID}.sbatch

## 2.2. Equilibration

In [ ]:
RUN_ID="NVT"
equi_prefix=step4.1_equilibration

./../../src/simulation_sbatch_script.sh --init_gro ${WORKDIR}/../../out/${sample}/${mini_prefix}/Run_EM/${mini_prefix}_EM.gro \
                                    --topol ${WORKDIR}/topol.top \
                                    --run_prefix ${equi_prefix} \
                                    --mdp ${WORKDIR}/step4.1_equilibration.mdp \
                                    --output_dir ${WORKDIR}/../../out/${sample}/${equi_prefix} \
                                    --sample ${sample} \
                                    --workdir ${WORKDIR} \
                                    --lab "gdwanglab" \
                                    --job_name "equil" \
                                    --run_duration "80:00:00" \
                                    --run ${RUN_ID} \
                                    --index ${WORKDIR}/index.ndx

sbatch ${WORKDIR}/${sample}_${equi_prefix}_Run_${RUN_ID}.sbatch

## 2.3. MD

In [ ]:
md_prefix="md"
equi_prefix="step4.1_equilibration"
for chain in chain1 chain2 chain3
do
    RUN_ID="MD_${chain}"

    ./../../src/simulation_sbatch_script.sh \
        --init_gro ${WORKDIR}/../../out/${sample}/${equi_prefix}/Run_NVT/${equi_prefix}_NVT.gro \
        --topol ${WORKDIR}/topol.top \
        --run_prefix ${md_prefix} \
        --mdp ${WORKDIR}/step5_production.mdp \
        --cpi ${WORKDIR}/../../out/${sample}/${md_prefix}/${RUN_ID}/${md_prefix}_${RUN_ID}.cpt \
        --output_dir ${WORKDIR}/../../out/${sample}/${md_prefix} \
        --sample ${sample} \
        --workdir ${WORKDIR} \
        --lab "gdwanglab" \
        --job_name "md" \
        --run_duration "250:00:00" \
        --run ${RUN_ID} \
        --index ${WORKDIR}/index.ndx

    sbatch ${WORKDIR}/${sample}_${md_prefix}_Run_${RUN_ID}.sbatch
done


## Analysis

In [ ]:
module purge
module load GCC/10.3.0
module load OpenMPI/4.1.1
module load GROMACS/2021.3-CUDA-11.3.1

In [ ]:
WORKDIR=/gpfs0/home1/gdwanglab/axk201/personal_projects/Pediatric_glioblastoma_singelcell/Notebook/Step17_Antibody_Molecular_Dynamics
cd ${WORKDIR}
find . -type f -name '#*#' -delete

### Center the trajectory

In [ ]:
for sample in 23C3_v1 Hu23C3_v1
do
    for md_chain in chain1 chain2 chain3
    do
        echo 0 0 | gmx trjconv \
            -f ${WORKDIR}/out/${sample}/md/Run_MD_${md_chain}/md_MD_${md_chain}.xtc \
            -s ${WORKDIR}/out/${sample}/md/Run_MD_${md_chain}/md_MD_${md_chain}.tpr \
            -o ${WORKDIR}/out/${sample}/md/Run_MD_${md_chain}/centered.xtc \
            -pbc mol \
            -skip 50 \
            -ur compact -center
    done
done


### Create index for protein chains

In [ ]:
printf "1 & a 1-4741\nname 16 SPP1\n1 & a 4742-8031\nname 17 Heavy\n1 & a 8032-11292\nname 18 Light\nq\n" | gmx make_ndx -f ./out/23C3_v1/md/Run_MD_chain1/md_MD_chain1.tpr -o ./in/23C3_v1/index_prot_chains.ndx
printf "1 & a 1-4741\nname 16 SPP1\n1 & a 4742-8047\nname 17 Heavy\n1 & a 8048-11317\nname 18 Light\nq\n" | gmx make_ndx -f ./out/Hu23C3_v1/md/Run_MD_chain1/md_MD_chain1.tpr -o ./in/Hu23C3_v1/index_prot_chains.ndx

In [ ]:
for sample in 23C3_v1 Hu23C3_v1
do
    for md_chain in chain1 chain2 chain3
    do
        for prot_chain in 16 17 18
        do 
            case $prot_chain in
                16) chain_name="SPP1" ;;
                17) chain_name="Heavy" ;;
                18) chain_name="Light" ;;
            esac

            echo ${prot_chain} | gmx rmsf \
                -s ${WORKDIR}/out/${sample}/md/Run_MD_${md_chain}/md_MD_${md_chain}.tpr \
                -f ${WORKDIR}/out/${sample}/md/Run_MD_${md_chain}/centered.xtc \
                -n ${WORKDIR}/in/${sample}/index_prot_chains.ndx -res \
                -e 50000 \
                -o ${WORKDIR}/out/${sample}/md/Run_MD_${md_chain}/rmsf_${sample}_${md_chain}_${chain_name}.xvg
        done
    done
done
